In [1]:
import cv2
import numpy as np

In [2]:
# h_min s_min v_min   h_max s_max v_max
myColors = [[72,22,0, 109,255,175]] # green
            # [103,102,0, 145,255,255]]  # blue
myColorVals = [[0,255,0]]
                # [255,0,0]]

In [3]:
def findColor(img, myColors):
    imgHSV = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    newPoints = []
    for i,color in enumerate(myColors):
        lower = np.array(color[:3])
        upper = np.array(color[3:])
        mask = cv2.inRange(imgHSV, lower, upper)
        x,y = getContours(mask)
        if x!=0 and y!=0:
            newPoints.append([x,y,i])
    return newPoints

In [4]:
def getContours(img):
    contours, hierarchy = cv2.findContours(img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    x,y,w,h=0,0,0,0
    for cnt in contours:
        area = cv2.contourArea(cnt)
        if area > 100:
            # cv2.drawContours(imgContour, cnt, -1, (255,0,0), 3)
            peri = cv2.arcLength(cnt, True)
            approx = cv2.approxPolyDP(cnt, 0.02*peri, True) #corner points
            x,y,w,h = cv2.boundingRect(approx)
    return x+w//2, y  # returning tip of bounding box

In [5]:
def drawPrevious(points, myColorVals):
    for (x,y,cid) in points:
        cv2.circle(imgContour, (x,y), 10, myColorVals[cid],cv2.FILLED)

In [13]:
cap = cv2.VideoCapture(0)
cap.set(3,640)      # width
cap.set(4,480)      # height
cap.set(10, 150)    # brightness

allPoints = []  # (x, y, colorID)

while True:
    success, img = cap.read()
    img = img[:,::-1,:]         # flipping horizontaly
    imgContour = img.copy()
    if not success or (cv2.waitKey(1) & 0xFF == ord('q')):
        if len(allPoints) > 0:
            allPoints = []
        else:
            break
    else:
        newPts = findColor(img, myColors)
        if len(newPts) !=0 :
            for pt in newPts:
                allPoints.append(pt)
        drawPrevious(allPoints, myColorVals)
        cv2.imshow("video", imgContour)
        # cv2.waitKey(10)
    
cap.release()
cv2.destroyAllWindows()

In [7]:
## color detection =======================================

def empty(p):
    pass

cap = cv2.VideoCapture(0)
cap.set(3,300)      # width
cap.set(4,300)      # height

cv2.namedWindow("TrackBars")
cv2.resizeWindow("TrackBars", 640, 480)
cv2.createTrackbar("hue min", "TrackBars", 0, 179, empty)
cv2.createTrackbar("hue max", "TrackBars", 179, 179, empty)
cv2.createTrackbar("sat min", "TrackBars", 0, 255, empty)
cv2.createTrackbar("sat max", "TrackBars", 255, 255, empty)
cv2.createTrackbar("val min", "TrackBars", 0, 255, empty)
cv2.createTrackbar("val max", "TrackBars", 255, 255, empty)

while True:
    success, img = cap.read()

    imgHSV = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    h_min = cv2.getTrackbarPos("hue min", "TrackBars")
    h_max = cv2.getTrackbarPos("hue max", "TrackBars")
    s_min = cv2.getTrackbarPos("sat min", "TrackBars")
    s_max = cv2.getTrackbarPos("sat max", "TrackBars")
    v_min = cv2.getTrackbarPos("val min", "TrackBars")
    v_max = cv2.getTrackbarPos("val max", "TrackBars")

    lower = np.array([h_min, s_min, v_min])
    upper = np.array([h_max, s_max, v_max])
    mask = cv2.inRange(imgHSV, lower, upper)

    # imgResult = cv2.bitwise_and(img, img, mask=mask)

    cv2.imshow("image", img)
    cv2.imshow("mask", mask)
    # cv2.imshow("result", imgResult)
    if (cv2.waitKey(200) & 0xFF == ord('q')):
        break    

cap.release()
cv2.destroyAllWindows()
